# RAG查询优化：多查询与查询转换技术
在基础RAG系统中，我们使用单一查询进行检索。但在实际应用中，用户的查询往往存在表达模糊、角度单一或过于笼统的问题。本章将介绍多种查询优化技术，让你的RAG系统能够更准确地理解用户意图。

```
# ❌ 单一查询的问题

用户查询: "机器学习是什么？"

# 可能错过的相关文档：
- "深度学习入门" (使用了不同但相关的术语)
- "AI算法基础" (更广泛的主题)
- "神经网络原理" (具体技术)
- "监督学习vs无监督学习" (细分话题)

# ✅ 查询优化后

原始查询: "机器学习是什么？"

生成的变体:
1. "什么是机器学习算法？"
2. "机器学习的基本概念和原理"
3. "AI中的机器学习技术"
4. "机器学习的应用场景"

→ 检索 → 合并结果 → 去重 → 生成答案
```
核心思路：**通过生成多个角度的查询、重写查询或分解复杂查询，增加检索到相关文档的概率。**


| **技术**            | **核心思想**  | **适用场景** | **复杂度** |
|-------------------|-----------|----------|---------|
| **Multi-Query**   | 生成查询的多个变体 | 用户查询表达不清 | ⭐       |
| **RAG-Fusion**    | 多查询+重排序融合 | 需要高质量结果  | ⭐⭐      |
| **Decomposition** | 分解复杂查询    | 多步骤问题    | ⭐⭐⭐     |
| **Step Back**     | 先问概括性问题   | 需要背景知识   | ⭐⭐      |
| **HyDE**          | 生成假设性文档   | 语义搜索增强   | ⭐⭐⭐     |

## 基础实现

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key=os.getenv("DASHSCOPE_API_KEY")

from langchain_qwq import ChatQwen
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 定义查询生成提示词
query_gen_prompt = ChatPromptTemplate.from_messages([
    ("system", """你是一个AI助手，负责生成查询的多个变体。
    给定一个用户查询，生成3个不同角度的相关查询。
    每行一个查询，不要编号。"""),
    ("human", "{question}")
])

llm = ChatQwen(
    model="qwen-plus",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    temperature=0.7  # 0 = 更精确, 1 = 更有创意
)

query_generator = (
    query_gen_prompt
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))  # 分割成列表
)

# 使用示例
original_query = "什么是Agent？"
queries = query_generator.invoke({"question": original_query})
print("原始查询:", original_query)
print("\n生成的查询变体:")
for i, q in enumerate(queries, 1):
    print(f"{i}. {q}")

原始查询: 什么是Agent？

生成的查询变体:
1. Agent在人工智能中指的是什么？  
2. Agent与普通程序的区别在哪里？  
3. 如何设计一个能够自主决策的Agent？


## Multi-Query 示例

优点 ✅：

1. 提高召回率（找到更多相关文档）
2. 覆盖不同角度和表达方式
3. 对用户表达不清的查询特别有效
4. 实现相对简单

缺点 ❌：

1. 增加检索成本（多次查询）
2. 可能引入噪声（不相关的变体）
3. 需要额外的LLM调用
4. 去重逻辑可能过滤掉有价值的文档

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key=os.getenv("DASHSCOPE_API_KEY")

